[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/102_fused.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/102_fused.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Mapping Overture Buildings and Foursquare Places with Leafmap + Fused**

Fetch data by calling [Fused UDFs](https://docs.fused.io/core-concepts/write/) and render it on a [Leafmap](https://leafmap.org/) map.

This notebook shows how you can:

1. Load [Overture Buildings](https://docs.overturemaps.org/getting-data/fused) for a drawn area
2. Load [Foursquare (FSQ) Place](https://docs.foursquare.com/data-products/docs/access-fsq-os-places) POIs for a drawn area

In [ ]:
# %pip install fused>=1.11.1 "leafmap[maplibre]" -q

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import fused
import leafmap
import leafmap.maplibregl as maplibre

# 1. Load Overture Buildings for drawn area

First draw a bounding box over an area of interest (AOI) and create a GeoDataFrame with it. Then, [run the Overture UDF](https://docs.fused.io/core-concepts/run/) to fetch data for that AOI.

## 1.1 First, draw an AOI

In [ ]:
m = leafmap.Map(center=[40.713370, -73.996181], zoom=14)
m

In [ ]:
# If no AOI is drawn, use the default AOI

if m.user_roi is None:
    m.user_roi = {'type': 'Feature',
 'properties': {},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-74.025621, 40.699967],
    [-74.025621, 40.730283],
    [-73.966055, 40.730283],
    [-73.966055, 40.699967],
    [-74.025621, 40.699967]]]}}

## 1.2 Then, convert it to a GeoDataFrame.

In [ ]:
import geopandas as gpd

# Convert drawing to GeoDataFrame
gdf_drawing = gpd.GeoDataFrame([m.user_roi['properties']], geometry=[shape(m.user_roi['geometry'])])

gdf_drawing

## 1.3 Fetch Overture Buildings for the AOI and render them on the map

The data comes from calling the "[Overture Maps Example](https://www.fused.io/workbench/catalog/Overture_Maps_Example-64071fb8-2c96-4015-adb9-596c3bac6787)" public UDF with `fused.run`.

In [ ]:
# Fetch Overture Bulidings GDF with Fused UDF
gdf = fused.run("UDF_Overture_Maps_Example", bbox=gdf_drawing)

In [ ]:
gdf.head()

In [ ]:
print(f"The number of buildings within the AOI: {len(gdf)}")

In [ ]:
# Add GDF to map
m.add_gdf(gdf[['id', 'geometry', 'height', 'class']], layer_type="fill", name="Buildings", fill_colors=["red"])

## 1.4 Visualize buildings in 3D.

In [ ]:
m = maplibre.Map(pitch=60)
m.add_basemap("USGS.USImagery")
gdf_filter = gdf[gdf.height > 0][['id', 'geometry', 'height', 'class']]
m.add_data(gdf_filter, column="height", cmap="Blues", extrude=True, name="Buildings")
m

In [ ]:
m.layer_interact()

# 2. Load FSQ Places POIs for drawn area

## 2.1 Fetch FSQ Places for the AOI (defined above) and render them on the map

The data comes from calling the "[Foursquare Open Source Places](https://www.fused.io/workbench/catalog/Foursquare_Open_Source_Places-5cd75ead-e319-4279-8efc-04276de145bc)" public UDF with `fused.run`.

In [ ]:
# Fetch FSQ GDF with Fused UDF
gdf = fused.run("UDF_Foursquare_Open_Source_Places", bbox=gdf_drawing)

# `add_points_from_xy` requires latitude and longitude columns
gdf['latitude'] = gdf.geometry.y
gdf['longitude'] = gdf.geometry.x

# Show place categories
set(gdf.level1_category_name.values)

In [ ]:
# Subsample the gdf
gdf_sample = gdf[['name', 'latitude', 'longitude', 'geometry']].head(200).reset_index(drop=True)
# gdf_sample = gdf[gdf.level1_category_name == 'Dining and Drinking'][['name', 'latitude', 'longitude']].head(1000).reset_index(drop=True)
gdf_sample

## 2.2 Visualize the POIs on a 2d map

In [ ]:
m = leafmap.Map(center=[40.713370, -73.996181], zoom=14)
# Add as marker cluster
m.add_points_from_xy(
    gdf_sample,
    x="longitude",
    y="latitude",
    spin=False,
    add_legend=True,
)
m

## 2.3 Visualize POIs with OpenStreetMap vector tiles

In [ ]:
m = maplibre.Map(pitch=60, style="https://maps.gishub.org/styles/openstreetmap.json")
m.add_gdf(gdf_sample, name="Places")
m

## 2.4 Visualize the POIs on a 3D map with Overture buildings

In [ ]:
m = maplibre.Map(zoom=18, pitch=60, style="positron")
m.add_overture_3d_buildings()
m.add_gdf(gdf_sample, name="Places")
m

## 2.5 Visualize the POIs on a 3D map with [OpenFreeMap](https://openfreemap.org/) vector tiles

In [ ]:
m = maplibre.Map(pitch=60, style="liberty")
m.add_gdf(gdf_sample, name="Places")
m